In [1]:
import yaml
import pandas as pd
from message_ix.utils import make_df
from message_ix.models import MESSAGE_ITEMS
from collections.abc import Mapping
from itertools import repeat

<IPython.core.display.Javascript object>

In [158]:
with open('tech_data_01_update.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)
    
years = [*range(700,730,10)]
regions = ['Westeros', 'Westerlands']

In [159]:
# scenario.regions is the list of nodes in the scenario
default_cost_by_region = {reg: 1 for reg in regions}
cost_by_region = {
    tech: tech_data['model_data']
            .get(tech,{})
            .get('cost_by_region', default_cost_by_region)
    for tech in set(tech_data) - set(['model_data'])
}

cost_by_region

{'DACCS': {'Westeros': 1, 'Westerlands': 1}}

In [160]:
# scenario.years is the list of years in the scenario
first_active_year = tech_data['model_data'].get('first_active_year')
default_costreduction_by_region = {'rate': 0}
costreduction_by_region = {
    tech: tech_data['model_data']
            .get(tech,{})
            .get('costreduction_by_region', default_costreduction_by_region)
    for tech in set(tech_data) - set(['model_data'])
}

vtg_years = [y for y in years if y >= first_active_year]
costreduction_by_region

{'DACCS': {'rate': 0}}

In [161]:
years_dict = {yv:[ya for ya in years if ya >= yv] for yv in years if yv >= first_active_year}

vtg = [yv for ya in list(years_dict.keys()) for yv in repeat(ya,len(years_dict[ya]))]

act = [yv for ya in list(years_dict.keys()) for yv in repeat(ya,len(years_dict[ya]))]

years_vtg_act = pd.DataFrame(
    {
    'year_vtg':[k for k, v in years_dict.items() for i in range(len(v))],
    'year_act':[ya for value in years_dict.values() for ya in value]
    }
)

years_vtg_act

,year_vtg,year_act
0,700,700
1,700,710
2,700,720
3,710,710
4,710,720
5,720,720


In [162]:
costperiodfactor_by_region = {
    tech: {
        y: (1-costreduction_by_region.get(tech).get('rate'))**(y-first_active_year)
        for y in years_vtg_act['year_vtg']
    }
    for tech in set(tech_data)-set(['model_data'])
}

costperiodfactor_by_region

{'DACCS': {700: 1, 710: 1, 720: 1}}

In [163]:
#par_dim = {'inv_cost':['node_loc'],'fix_cost':['node_loc'],'var_cost':['node_loc']}
parameters = {}
for tech in set(tech_data) - set(['model_data']):
    parameters.update({par: list(MESSAGE_ITEMS[par]['idx_names']) for par in set(tech_data[tech])})
data = {par: [] for par in list(parameters.keys())}

In [164]:
print(MESSAGE_ITEMS['technical_lifetime']['idx_names'],'-- technical_lifetime')
print(MESSAGE_ITEMS['growth_new_capacity_up']['idx_names'],'-- growth_new_capacity_up')
print(MESSAGE_ITEMS['initial_new_capacity_up']['idx_names'],'-- initial_new_capacity_up')
print(MESSAGE_ITEMS['inv_cost']['idx_names'],'-- inv_cost')
print('')
print(MESSAGE_ITEMS['fix_cost']['idx_names'],'-- fix_cost')
print(MESSAGE_ITEMS['capacity_factor']['idx_names'],'-- capacity_factor')
print('')
print(MESSAGE_ITEMS['var_cost']['idx_names'],'-- var_cost')
print(MESSAGE_ITEMS['emission_factor']['idx_names'],'-- emission_factor')
print('')
print(MESSAGE_ITEMS['input']['idx_names'],'-- input')


('node_loc', 'technology', 'year_vtg') -- technical_lifetime
('node_loc', 'technology', 'year_vtg') -- growth_new_capacity_up
('node_loc', 'technology', 'year_vtg') -- initial_new_capacity_up
('node_loc', 'technology', 'year_vtg') -- inv_cost

('node_loc', 'technology', 'year_vtg', 'year_act') -- fix_cost
('node_loc', 'technology', 'year_vtg', 'year_act', 'time') -- capacity_factor

('node_loc', 'technology', 'year_vtg', 'year_act', 'mode', 'time') -- var_cost
('node_loc', 'technology', 'year_vtg', 'year_act', 'mode', 'emission') -- emission_factor

('node_loc', 'technology', 'year_vtg', 'year_act', 'mode', 'node_origin', 'commodity', 'level', 'time', 'time_origin') -- input


In [165]:
# Basic DataFrame
group1 = ['inv_cost','technical_lifetime','growth_new_capacity_up','initial_new_capacity_up']
group2 = ['fix_cost','capacity_factor']
for tech, par_dict in tech_data.items():
    if tech != 'model_data':
        for par, par_data in par_dict.items():
            print(par)
            if not isinstance(par_data, Mapping):
                par_data = {'value': par_data, 'unit': '-'}
            if MESSAGE_ITEMS[par]['idx_names'] == ('node_loc', 'technology', 'year_vtg'):
                kwargs = {'year_vtg': sorted(list(set(years_vtg_act['year_vtg'])))}
            elif all(e in ('node_loc', 'technology', 'year_vtg', 'year_act') for e in MESSAGE_ITEMS[par]['idx_names']):
                kwargs = {'year_vtg': years_vtg_act['year_vtg'],
                          'year_act': years_vtg_act['year_act']}
            #if 'mode' in parameters[par]:
            #    kwargs.update({'mode':par_data['mode']})
            #if 'emission' in parameters[par]:
            #    kwargs.update({'emission':par_data['emission']})
            data[par].append(
                make_df(
                    par,
                    technology=tech,
                    value=par_data['value'],
                    unit=par_data['unit'],
                    **kwargs,
                )
            )

data = {k: pd.concat(v) for k, v in data.items()}

data

inv_cost
fix_cost
var_cost
emission_factor
capacity_factor
technical_lifetime
growth_new_capacity_up
initial_new_capacity_up


{'fix_cost':   node_loc technology  year_vtg  year_act  value   unit
 0     None      DACCS       700       700     10  $/kWa
 1     None      DACCS       700       710     10  $/kWa
 2     None      DACCS       700       720     10  $/kWa
 3     None      DACCS       710       710     10  $/kWa
 4     None      DACCS       710       720     10  $/kWa
 5     None      DACCS       720       720     10  $/kWa,
 'growth_new_capacity_up':   node_loc technology  year_vtg  value unit
 0     None      DACCS       700    0.5    -
 1     None      DACCS       710    0.5    -
 2     None      DACCS       720    0.5    -,
 'inv_cost':   node_loc technology  year_vtg  value  unit
 0     None      DACCS       700   2500  $/kW
 1     None      DACCS       710   2500  $/kW
 2     None      DACCS       720   2500  $/kW,
 'var_cost':   node_loc technology  year_vtg  year_act  mode  time  value   unit
 0     None      DACCS       700       700  None  None    100  $/kWa
 1     None      DACCS       700  

In [166]:
# Expanded DataFrame
data_expand ={par: [] for par in list(parameters.keys())} 
for par in list(parameters.keys()):
    if par in group1+group2:
        for tech, diffs in tech_data['model_data'].items():
            if tech != 'first_active_year':
                for reg in regions:
                    multiplier = []
                    for y in data[par]['year_vtg'].values:
                        multiplier.append(diffs.get(par,{}).get('node_loc',{}).get(reg,1))
                                          *diffs.get(par).get(year_vtg).get(reg)
                                          
#                                               *costperiodfactor_by_region.get(tech).get(y))
                    value = data[par]['value']*multiplier
                    data_expand[par].append(
                        data[par].assign(node_loc=reg,value=value)
                       )
                    
data_expand

{'fix_cost': [   node_loc technology  year_vtg  year_act  value   unit
  0  Westeros      DACCS       700       700     10  $/kWa
  1  Westeros      DACCS       700       710     10  $/kWa
  2  Westeros      DACCS       700       720     10  $/kWa
  3  Westeros      DACCS       710       710     10  $/kWa
  4  Westeros      DACCS       710       720     10  $/kWa
  5  Westeros      DACCS       720       720     10  $/kWa,
        node_loc technology  year_vtg  year_act  value   unit
  0  Westerlands      DACCS       700       700     10  $/kWa
  1  Westerlands      DACCS       700       710     10  $/kWa
  2  Westerlands      DACCS       700       720     10  $/kWa
  3  Westerlands      DACCS       710       710     10  $/kWa
  4  Westerlands      DACCS       710       720     10  $/kWa
  5  Westerlands      DACCS       720       720     10  $/kWa],
 'growth_new_capacity_up': [   node_loc technology  year_vtg  value unit
  0  Westeros      DACCS       700    0.5    -
  1  Westeros     

In [61]:
data_expand = {k: pd.concat(v) for k, v in data_expand.items() 
               if k in group2+group1}

In [62]:
with pd.ExcelWriter('tech_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data_expand.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [79]:
tech_data

{'DACCS': {'inv_cost': {'value': 2500, 'unit': '$/kW'},
  'fix_cost': {'value': 10, 'unit': '$/kWa'},
  'technical_lifetime': {'value': 20, 'unit': 'y'},
  'growth_new_capacity_up': {'value': 0.5, 'unit': '-'},
  'initial_new_capacity_up': {'value': 0.5, 'unit': 'GW'}},
 'model_data': {'DACCS': {'cost_by_region': {'Westeros': 1,
    'Westerlands': 0.8},
   'costreduction_by_region': {'rate': 0.05}},
  'first_active_year': 700}}

In [114]:
tech_data

{'DACCS': {'inv_cost': {'value': 2500, 'unit': '$/kW'},
  'fix_cost': {'value': 10, 'unit': '$/kWa'},
  'emission_factor': {'value': -20,
   'unit': 'tCO2/kWa',
   'mode': 'standard',
   'emission': 'CH4'},
  'technical_lifetime': {'value': 20, 'unit': 'y'},
  'growth_new_capacity_up': {'value': 0.5, 'unit': '-'},
  'initial_new_capacity_up': {'value': 0.5, 'unit': 'GW'}},
 'model_data': {'DACCS': {'cost_by_region': {'Westeros': 1,
    'Westerlands': 0.8},
   'costreduction_by_region': {'rate': 0.05}},
  'first_active_year': 700}}